In [ ]:
# Promoter data mining

import numpy as np
import pandas as pd

# 1. Load the whole genome sequence of cyanobacteria

pccgene = []
pccseq = ''
data = open('Whole_genome_of_synechocystis.txt','r') # Whole genmoe sequece of Synechocyntis sp. PCC6803
line = data.readlines()
gspoint = line.index('ORIGIN\n')
for i in range(len(line)-gspoint-1):
    templist = line[i+gspoint].strip().split()[1:]
    for partseq in templist:
        pccseq = pccseq+partseq
data.close()

pccseq = pccseq.upper()

# 2. Convert differential RNA-sequencing data to training datasset

trlist = []
comp_dic = {'A':'T','G':'C','C':'G','T':'A'}
prolen = 50
df = pd.read_excel('Differential_RNA_sequencing_data.xlsx',header = 1,sheet_name = 'S1 - All TUs')
numpydf = df.to_numpy()
for i in numpydf:
    if i[0] == 'Chr':
        if i[4] == '+':
            trlist.append((pccseq[i[2]-(prolen+1):i[2]-1],i[19]))
        else:
            compseq = ''
            seq = pccseq[i[2]:i[2]+prolen]
            for s in seq:
                compseq +=comp_dic[s]
            revcomp_seq = compseq[::-1]
            trlist.append((revcomp_seq, i[19]))
    else:
        break

# 3. Save training dataset

from openpyxl import Workbook

write_wb = Workbook()

write_ws = write_wb.active

write_ws['A1'] = 'Promoter'
write_ws['B1'] = 'Reads'

for i in range(len(trlist)):
    write_ws.cell(i+2,1,trlist[i][0])
    write_ws.cell(i+2,2,trlist[i][1])

write_wb.save('Training dataset.xlsx')